# Goede Voornemens het hele jaar door
Bespreking API backend - Python AWS Lambda CloudFunctions. 

## Één router en vijf API's

1. lambda_handler, 2. sessionmake, 3. sessionread, 4. userread, 5. userwrite, 6. usersread

De code van deze zes functies wordt hieronder in code blokken getoond maar kan niet gedraaid worden. Daarvoor ontbreekt de AWS Environment. Deze AWS environment levert requests af aan de lambda_handler en stuurt responses terug. Ook is Cloud Storage efficiënt benaderbaar met de boto3 module.


### Router - lambda_handler

De router kijkt welke variabelen er in de request worden aangeleverd en "beslist" op basis daarvan welke API functie er aangeroepen moet worden. De response de API functie wordt dan weer doorgestuurd naar de client. Leuk feitje: de Pythons requests functie levert base64 versleutelde requests. De Web Fetch levert "ruwe" utf-8 data. De handler moet een decoder bevatten.

In [2]:

def lambda_handler(event, context):

    # print("event", event)

    req = {}
    
    if 'body' in list( event.keys()):
        
        req = event['body']
        
        if 'isBase64Encoded' in list( event.keys()) and event["isBase64Encoded"]:
            req = base64.b64decode(event['body']).decode('utf-8')
            req = json.loads( req )
            req = req["body"]

        elif type(req) is not dict:
            req = json.loads( event['body'] )

    print("req", req)

    reqkeys = list( req.keys() )
    if "loginname" in reqkeys:
        
        if req["loginname"]=="session":
            if "wwdwsession" in reqkeys:
                resjson = usermethods.sessionread(req["wwdwsession"] )
            else:
                resjson = {}
        else:
            resjson = usermethods.sessionmake(req["loginname"] )
            
    elif "article" in reqkeys:
        resjson = usermethods.userwrite( req )

    elif "documenthash" in reqkeys:
        resjson = usermethods.userread( req["documenthash"] )

    elif "usersmake" in reqkeys:
        resjson = usersmethods.usersmake()

    else:
        resjson = usersmethods.usersread()


    return {
        "statusCode": 200,
        "headers": {
            "Content-Type": "application/json"
        },
        "body": json.dumps( resjson  )
    }
    


### Session make

De sessionmake API genereert een sessionid en koppelt dit aan een userid. Als een user bijvoorbeeld zijn gegevens wil wijzigen dient deze het sessionid en de te wijzigen data aan te leveren. De wijziging wordt dan doorgevoerd op het userid dat in het bestand sessions.json is gekoppeld aan de een sessionid. Als er nog geen user bestaat voor de loginnaam dan maakt sessionmake tevens een nieuwe (lege) user aan.

In [3]:
def sessionmake(loginname):

    nowdtm  = datetime.today().strftime('%Y%m%d')

    # allways a new session
    wwdwsession = nowdtm + "".join((random.choice("abcdefghij0123456789") for i in range(8)))
    
    object_content = s3.Object("wwdw", "users.json")
    file_content = object_content.get()['Body'].read().decode('utf-8')
    users = json.loads(file_content)
    
    user = list(filter(lambda u: u['loginname'] == loginname, users))    
    
    if(len(user)==1):
        user = user[0]
        wwdwid = user["wwdwid"]
        uservoornemens = userread(wwdwid)

    else:
        # write new wwdwuid        
        wwdwid = "wwdwid" + nowdtm + "".join((random.choice("abcdefghij0123456789") for i in range(4)))
        uservoornemens = {
            "username": "",
            "loginname": loginname,
            "wwdwid": wwdwid,
            "voornemen": [{
                    "text": "",
                    "step":[{ "text": "" }],
                    "thought":[{ "text": ""}]
                }]
            }
        s3object = s3.Object('wwdw', wwdwid+'.json')
        s3object.put( Body = ( bytes( json.dumps( uservoornemens ).encode('UTF-8') ) ) )    

        # append new wwdwuid to users
        del uservoornemens["voornemen"]
        users.append( uservoornemens )
        s3object = s3.Object('wwdw', 'users.json')
        s3object.put( Body = ( bytes( json.dumps( users ).encode('UTF-8') ) ) )    

    uservoornemens["wwdwsession"] = wwdwsession

    # upsert sessions {}
    object_content = s3.Object("wwdw", "sessions.json")
    file_content = object_content.get()['Body'].read().decode('utf-8')
    sessions = json.loads(file_content)

    sessions[wwdwsession] = {"wwdwid": wwdwid}
    s3object = s3.Object('wwdw', 'sessions.json')
    s3object.put( Body = ( bytes( json.dumps( sessions ).encode('UTF-8') ) ) )    

    
    print("session for", loginname, wwdwid, uservoornemens["wwdwsession"] )

    del uservoornemens["loginname"]

    return uservoornemens
    
    

### Session read

Op de client wordt het sessionid opgeslagen in een eeuwig durend cookie. Dit cookie wordt meegestuurd met elk request en indien nodig gebruikt. Als de [live demo](https://jhmj-io.github.io/ba-wk2201-wwdw/) opnieuw wordt aangeroepen wordt bij de onload het sessionid uit het cookie op de server geverifieerd. Als het bestaat dan is de user ingelogd.

In [ ]:
def sessionread(wwdwsession):
    object_content = s3.Object("wwdw", "sessions.json")
    file_content = object_content.get()['Body'].read().decode('utf-8')
    sessions = json.loads(file_content)
    if wwdwsession in list(sessions.keys()):
        print("sessionread", wwdwsession, sessions[wwdwsession])
        uservoornemens = userread( sessions[wwdwsession]["wwdwid"] )

        del uservoornemens["loginname"]

        return uservoornemens

    else:
        return {} 

### User read

Opvragen van de voornemens data van een user aan de hand van zijn userid.


In [ ]:
def userread( wwdwid ):
    
    object_content = s3.Object("wwdw", wwdwid + ".json" )
    file_content = object_content.get()['Body'].read().decode('utf-8')
    usermapped = json.loads(file_content)
    usermapped["wwdwid"] = wwdwid # should already be in dict

    return usermapped



Bovenstaande functie levert de response op onderstaand request. Voor de goede orde: het request wordt normaal gesproken uitgevoerd door Web Fetch -in een browser- en de response komt dus van de userread die draait in de AWS Lambda Cloud Function achter: https://8lgmayxgl6.execute-api.eu-central-1.amazonaws.com/default/wwdw

In [5]:
import json
import requests

cloudfunction = 'https://8lgmayxgl6.execute-api.eu-central-1.amazonaws.com/default/wwdw'

req = {
  "body": {
    "documenthash": "wwdwid202201072841"
   }
}

vdata = requests.post(cloudfunction, data=json.dumps(req))

vdata.json()


{'username': 'Sander!',
 'loginname': 'sander',
 'wwdwid': 'wwdwid202201072841',
 'voornemen': [{'text': 'Meer piano spelen',
   'step': [{'text': 'Gewoon doen'}],
   'thought': [{'text': 'Bit Academy is ook leuk'}]}]}

### User write

Opslaan van al dan niet gewijzgde data in het voor elke gebruiker unieke userid.json bestand in Cloud Storage. De data wordt alleen opgeslagen als er een bestaand sessionid wordt meegeleverd. Hackers opgelet: je kan een sessionid vinden in het eeuwig durende cookie en dit dan tevens inzetten op andere computers. Je kan weliswaar alleen de data van de gebruiker gekoppeld aan die session er mee wijzigen.

In [ ]:
def userwrite( req ):

    wwdwsession = req["wwdwsession"]
    article = req["article"]
    
    object_content = s3.Object("wwdw", "sessions.json")
    file_content = object_content.get()['Body'].read().decode('utf-8')
    sessions = json.loads(file_content)
    
    if wwdwsession not in list(sessions.keys()):
        return {"articlewrite": "fail" } 
        
    print("uservoornemens article", article )

    uservoornemens = userread( sessions[wwdwsession]["wwdwid"] )
    print("uservoornemens s3", uservoornemens )

    uservoornemens["username"]  = article["username"]
    uservoornemens["voornemen"] = article["voornemen"]

    wwdwid = sessions[wwdwsession]["wwdwid"]


    s3object = s3.Object('wwdw', wwdwid+'.json')
    s3object.put( Body = ( bytes( json.dumps( uservoornemens ).encode('UTF-8') ) ) )    

    # update username in users
    object_content = s3.Object("wwdw", "users.json")
    file_content = object_content.get()['Body'].read().decode('utf-8')
    users = json.loads(file_content)
    
    useri = [i for i, d in enumerate(users) if d["wwdwid"]==wwdwid ][0]
    users[useri]["username"] = article["username"]
    
    s3object = s3.Object('wwdw', 'users.json')
    s3object.put( Body = ( bytes( json.dumps( users ).encode('UTF-8') ) ) )    

    return {"articlewrite": "OK"}

Onderstaand een write request dat door bovenstaande functie wordt uitgevoerd.

In [6]:
cloudfunction = 'https://8lgmayxgl6.execute-api.eu-central-1.amazonaws.com/default/wwdw'

req = {
  "body": {
    "wwdwsession": "bla-die-bla",
    "article": {
      "username": "Sander!",
      "voornemen": [
        {
          "text": "Meer muziek",
          "step": [{"text": "Kan ook luisten"}],
          "thought": [{"text": "Gaat samen met Bit Academy"}]
        }
      ]
    }
   }
}

vdata = requests.post(cloudfunction, data=json.dumps(req))

vdata.json()

{'articlewrite': 'fail'}

### Users read

Op de home page wordt een lijst gegeven van de waaghalzen die op de [live demo](https://jhmj-io.github.io/ba-wk2201-wwdw/) hun goede voornemens bespreken.

In [ ]:
def usersread():

    object_content = s3.Object("wwdw", "users.json")

    file_content = object_content.get()['Body'].read().decode('utf-8')

    json_content = json.loads(file_content)
    
    usersmapped = list( map(lambda u: {"username": u["username"], "wwdwid": u["wwdwid"]}, json_content) )

    return usersmapped
    

Onderstaand request vraagt om de user lijst. 

In [7]:
cloudfunction = 'https://8lgmayxgl6.execute-api.eu-central-1.amazonaws.com/default/wwdw'

req = {
  "body": {
   }
}

vdata = requests.post(cloudfunction, data=json.dumps(req))

vdata.json()

[{'username': 'Joep!', 'wwdwid': 'wwdwid2022010780bh'},
 {'username': 'Sander!', 'wwdwid': 'wwdwid202201072841'},
 {'username': 'Coach', 'wwdwid': 'wwdwid20220107ahb5'}]